In [1]:
import pandas as pd
import geopandas as geo
import numpy as np

In [23]:
columns_needed = [
"network_type",
"lat",
"long",
"loc_accuracy",
"country_location",
"download_kbit",
"upload_kbit",
 "implausible"   
]

In [44]:
df = pd.read_csv("Daten/rtr_datensatz_2019_2020_komplett.csv",usecols=columns_needed, sep=",")

In [12]:
rtr.network_type.value_counts()

WLAN            2017536
LAN             1537839
LTE              694672
HSPA+             50141
CELLULAR_ANY      42678
NR                 8640
3G/4G              7561
EDGE               7017
HSDPA              4873
NR NSA             4806
LTE NR avail       3887
UMTS               3565
Ethernet           1062
HSPA                596
2G/4G               512
LTE CA              467
2G/3G               440
HSUPA               233
2G/3G/4G            115
EVDO_A               76
IWLAN                56
GSM                  32
Bluetooth            29
EHRPD                 7
1xRTT                 3
Name: network_type, dtype: int64

In [61]:
Types_mobile =["LTE", "HSPA+","CELLULAR_ANY"]
Types =["WLAN","LAN","LTE","HSPA+","CELLULAR_ANY"]
Types_festnetz = ["WLAN","LAN"]

In [45]:
df = df[(df["loc_accuracy"] < 150 ) & (df["country_location"] == "AT") #wie gross raster? 100 ok?
        & (df["implausible"] == False) & df["network_type"].isin(Types) ]

In [37]:
df.network_type.value_counts()

WLAN            4432
LAN             2356
LTE             1333
HSPA+             54
CELLULAR_ANY      18
Name: network_type, dtype: int64

In [46]:
df = df.drop(labels=['loc_accuracy', 'country_location', 'implausible'], axis=1)

In [47]:
gdf = geo.GeoDataFrame(
    df, geometry=geo.points_from_xy(df.long,df.lat),crs="EPSG:4326")
gdf = gdf.to_crs("EPSG:31287")
gdf = gdf.drop(labels=['lat', 'long'], axis=1)

In [48]:
gdf.reset_index(drop=True,inplace=True)

In [42]:
AUTmap = geo.read_file("Atlas Daten/STATISTIK_AUSTRIA_L000100_LAEA.shp",crs= "EPSG:3035")
# EPSG:3035 Projektion

In [50]:
AUTmap= AUTmap.to_crs("EPSG:31287")

In [51]:
AUTmap_Merged = geo.sjoin(AUTmap,gdf,how="left")

In [62]:
AUTmap_Merged_mobile = AUTmap_Merged[AUTmap_Merged["network_type"].isin(Types_mobile)]
AUTmap_Merged_festnetz = AUTmap_Merged[AUTmap_Merged["network_type"].isin(Types_festnetz)]


,ID,NAME,geometry,index_right,network_type,download_kbit,upload_kbit
391,100mN27250E48014,CRS3035RES100mN2725000E4801400,"POLYGON ((629290.726 399235.890, 629295.160 39...",2179018.0,LTE,54592.0,35228.0
764,100mN27219E48041,CRS3035RES100mN2721900E4804100,"POLYGON ((631849.573 396013.133, 631854.005 39...",1130142.0,CELLULAR_ANY,33116.0,11163.0
791,100mN27217E48044,CRS3035RES100mN2721700E4804400,"POLYGON ((632140.295 395799.291, 632144.727 39...",217667.0,CELLULAR_ANY,17786.0,8433.0
791,100mN27217E48044,CRS3035RES100mN2721700E4804400,"POLYGON ((632140.295 395799.291, 632144.727 39...",2008796.0,CELLULAR_ANY,8462.0,9788.0
791,100mN27217E48044,CRS3035RES100mN2721700E4804400,"POLYGON ((632140.295 395799.291, 632144.727 39...",2029271.0,CELLULAR_ANY,9181.0,6327.0
...,...,...,...,...,...,...,...
8434557,100mN27257E48012,CRS3035RES100mN2725700E4801200,"POLYGON ((629122.044 399944.330, 629126.479 40...",2341128.0,LTE,37523.0,10806.0
8434557,100mN27257E48012,CRS3035RES100mN2725700E4801200,"POLYGON ((629122.044 399944.330, 629126.479 40...",1156388.0,LTE,58197.0,24626.0
8434557,100mN27257E48012,CRS3035RES100mN2725700E4801200,"POLYGON ((629122.044 399944.330, 629126.479 40...",1156369.0,LTE,36169.0,26518.0
8434557,100mN27257E48012,CRS3035RES100mN2725700E4801200,"POLYGON ((629122.044 399944.330, 629126.479 40...",1174979.0,LTE,104710.0,27814.0


In [65]:
AUTmap_Merged_mobile['download_kbit'] = AUTmap_Merged_mobile['download_kbit'].fillna(0)
AUTmap_Merged_festnetz['download_kbit'] = AUTmap_Merged_festnetz['download_kbit'].fillna(0)

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [69]:
AUTmap_Merged_mobile["Download_Mean"] = AUTmap_Merged_mobile.groupby('ID')["download_kbit"].transform('mean')
AUTmap_Merged_festnetz["Download_Mean"] = AUTmap_Merged_festnetz.groupby('ID')["download_kbit"].transform('mean')

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [70]:
AUTmap_Merged_mobile["Download_Max"] = AUTmap_Merged_mobile.groupby('ID')["download_kbit"].transform('max')
AUTmap_Merged_festnetz["Download_Max"] = AUTmap_Merged_festnetz.groupby('ID')["download_kbit"].transform('max')

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [71]:
AUTmap_Merged_mobile["Download_Median"] = AUTmap_Merged_mobile.groupby('ID')["download_kbit"].transform('median')
AUTmap_Merged_festnetz["Download_Median"] = AUTmap_Merged_festnetz.groupby('ID')["download_kbit"].transform('median')

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [73]:
AUTmap_Merged_mobile["Download_Count"] = AUTmap_Merged_mobile.groupby('ID')["download_kbit"].transform('count')
AUTmap_Merged_festnetz["Download_Count"] = AUTmap_Merged_festnetz.groupby('ID')["download_kbit"].transform('count')

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [83]:
df_gov_fest = pd.read_csv("Atlas Daten/festnetz_2021q3_20220203.csv", sep=",")

In [90]:
df_gov_fest_spatial = AUTmap.merge(df_gov_fest,  left_on="ID", right_on="l000100v3")

In [ ]:
#count plotten!"

In [92]:
df_gov_mobil = pd.read_csv("Atlas Daten/mobilfunknetz_2021q3_20220203.csv", sep=",")

In [93]:
df_gov_mobil_spatial = AUTmap.merge(df_gov_mobil, left_on="ID", right_on="l000100v3")

In [94]:
#  Mbit/s -> kBit/s
df_gov_mobil["download"] = 1000 * df_gov_mobil["download"]
df_gov_mobil["upload"] = 1000 * df_gov_mobil["upload"]
df_gov_mobil_spatial["download"] = 1000 * df_gov_mobil_spatial["download"]
df_gov_mobil_spatial["upload"] = 1000 * df_gov_mobil_spatial["upload"]

In [84]:
AUTmap_Merged_mobile.to_file('AUTmap_Merged_mobile.gpkg', driver='GPKG', layer='Mobil_RTR')  


C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [85]:
AUTmap_Merged_festnetz.to_file('AUTmap_Merged_festnetz.gpkg', driver='GPKG', layer='Festnetz_RTR')  

In [98]:
df_gov_fest_spatial.to_file('festnetz_gov.gpkg', driver='GPKG', layer='Festnetz_Gov')  


C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [ ]:
df_gov_mobil_spatial.to_file('festnetz_mobil.gpkg', driver='GPKG', layer='Mobil_Gov')  


NameError: name 'geo' is not defined